In [1]:
import os

In [2]:
%pwd

'F:\\Question-Answering-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'F:\\Question-Answering-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    train_data_path: Path
    valid_data_path: Path
    tokenizer_name: Path

In [6]:
from Question_Answering.constants import *
from Question_Answering.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            valid_data_path = config.valid_data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config

In [8]:
import os
from Question_Answering.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

F:\Question-Answering-Project\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [103]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)
        self.max_length = MAX_LENGTH
        self.stride = STRIDE

    def preprocess_training_data(self, examples):
        questions = [q.strip() for q in examples["question"]]
        inputs = self.tokenizer(
            questions,
            examples["context"],
            max_length=self.max_length,
            truncation="only_second",
            stride=self.stride,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            padding="max_length",
        )
        offset_mapping = inputs.pop("offset_mapping")
        sample_map = inputs.pop("overflow_to_sample_mapping")
        answers = examples["answers"]
        start_positions = []
        end_positions = []

        for i, offset in enumerate(offset_mapping):
            sample_idx = sample_map[i]
            answer = answers[sample_idx]
            start_char = answer["answer_start"][0]
            end_char = answer["answer_start"][0] + len(answer["text"][0])
            sequence_ids = inputs.sequence_ids(i)

            # Find the start and end of the context
            idx = 0
            while sequence_ids[idx] != 1:
                idx += 1
            context_start = idx
            while sequence_ids[idx] == 1:
                idx += 1
            context_end = idx - 1

            # If the answer is not fully inside the context, label is (0, 0)
            if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
                start_positions.append(0)
                end_positions.append(0)
            else:
                # Otherwise it's the start and end token positions
                idx = context_start
                while idx <= context_end and offset[idx][0] <= start_char:
                    idx += 1
                start_positions.append(idx - 1)

                idx = context_end
                while idx >= context_start and offset[idx][1] >= end_char:
                    idx -= 1
                end_positions.append(idx + 1)

            inputs["start_positions"] = start_positions
            inputs["end_positions"] = end_positions
    
        return {
            'input_ids' : inputs['input_ids'],
            'token_type_ids' : inputs['token_type_ids'],
            'attention_mask': inputs['attention_mask'],
            'start_positions': inputs["start_positions"],
            'end_positions': inputs["end_positions"]
        }
    

    def preprocess_validation_data(self, examples):
        questions = [q.strip() for q in examples["question"]]
        inputs = self.tokenizer(
            questions,
            examples["context"],
            max_length=self.max_length,
            truncation="only_second",
            stride=self.stride,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            padding="max_length",
        )

        sample_map = inputs.pop("overflow_to_sample_mapping")
        example_ids = []

        for i in range(len(inputs["input_ids"])):
            sample_idx = sample_map[i]
            example_ids.append(examples["id"][sample_idx])

            sequence_ids = inputs.sequence_ids(i)
            offset = inputs["offset_mapping"][i]
            inputs["offset_mapping"][i] = [
                o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
            ]

        inputs["example_id"] = example_ids
        return {
            'input_ids' : inputs['input_ids'],
            'token_type_ids' : inputs['token_type_ids'],
            'attention_mask': inputs['attention_mask'],
            'offset_mapping': inputs["offset_mapping"],
            'example_id': inputs["example_id"]
        }

    def convert(self):
        train_dataset = load_from_disk(self.config.train_data_path)
        preprocessed_train_dataset = train_dataset.map(self.preprocess_training_data, batched = True, remove_columns=train_dataset.column_names)
        preprocessed_train_dataset.save_to_disk(os.path.join(self.config.root_dir,"train_dataset"))
        valid_dataset = load_from_disk(self.config.valid_data_path)
        preprocessed_valid_dataset = valid_dataset.map(self.preprocess_validation_data, batched = True, remove_columns=valid_dataset.column_names)
        preprocessed_valid_dataset.save_to_disk(os.path.join(self.config.root_dir,"validation_dataset"))
        

In [95]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2023-11-02 22:23:39,408: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-02 22:23:39,411: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-02 22:23:39,413: INFO: common: created directory at: artifacts]
[2023-11-02 22:23:39,416: INFO: common: created directory at: artifacts/data_transformation]


Saving the dataset (1/1 shards): 100%|██████████████████████████████████████████████████████████████████████████| 10822/10822 [00:00<00:00, 23539.77 examples/s]
